# Analysis of Mislabeled, swapped data


This jupyter notebook analyzes the trees obtained from mislabelled (aka swapped) sequences. The sequences are from the simulated dataset. Mislabaling a paralog is equivalent to getting two paralogs and swap them while keeping their label (aka gene/paralog-ID) untouched. This will generate as a consequence erroneus orthologs assignments. Since it is the simulated dataset ortholog assignment are known a priori and are ground trouth, to simulate a real world scenario wHere there is more confusion and mistakes about orthologs assignments, we decided to swap some sequences to create wrong ortholog sets. 
<br><br>
The swapping happens in two steps for each family in the simulated dataset (500): first select the species to do the swapping in, then among the paralogs in that species actually swap the desired number of sequences around. Both species selection and sequence selection is random. So the swapping combination (10, 9) refers to selcting 10 species in which 9 sequences will be swapped. The 9 sequences are swapped among themselves.


## Table of Contents
- [1. Chapter 1 - Introduction](#chapter-1-introduction)
- [2. Chapter 2 - R packages](#chapter-2-R-packages)
- [3. Chapter 3 - Input specification](#chapter-3-Input-specification)
- [4. Chapter 4 - Swapped label vs true topologies](#chapter-4-Swapped-label-vs-true-topologies)
- [5. Chapter 5 - RF tables report](#chapter-5-RF-tables-report)

<a id="chapter-1-introduction"></a>

## Chapter 1 - Introduction

TODO add description of notebook and how to use it.


<a id="chapter-2-R-packages"></a>

## Chapter 2 - R packages

Here is the list of packages needed for this notebook.

In [1]:
library("phangorn")
library("ggplot2")
library("reshape2")
library("geiger")
library("adephylo")
library("phytools")
library("rlist")
library("plotrix")
library("plyr")
library("dplyr")
library("hrbrthemes")
library("rstatix")
library("gplots")
library("viridis")
library('gtools')
library(gridExtra)
library(grid)
library("ggforce")
library("scales")
library("pROC")
library("tidyr")
library("stringr")
library("patchwork")
library("ggarchery")

Loading required package: ape

Loading required package: phytools

Loading required package: maps

Loading required package: ade4


Attaching package: ‘plotrix’


The following object is masked from ‘package:phytools’:

    rescale



Attaching package: ‘plyr’


The following object is masked from ‘package:maps’:

    ozone



Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following object is masked from ‘package:ape’:

    where


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘rstatix’


The following objects are masked from ‘package:plyr’:

    desc, mutate


The following object is masked from ‘package:stats’:

    filter



Attaching package: ‘gplots’


The following object is masked from ‘package:plotrix’:

    plotC

<a id="chapter-3-Input-specification"></a>

## Chapter 3 - Input specification

Change the parent_of_all_dir variable to give the location of the input. All input files are expected to be under it. Otherwise change the single input location to you liking.<br>
All input files or directory paths are listed here.

In [15]:
# the location of the directory that contains all inputs for analyses 
parent_of_all_dir = "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/"

# The directory (one or more) that contain all the results from the main.nf pipeline run. It can be a glob path.
dir_containing_all_pipe_results = paste0(parent_of_all_dir, "rerun_25_species_corrected_branches/swapped_data/group*")

# The file containing the rename mapping of the reference paralog tree (generic)
reference_tree_codefile = file.path(parent_of_all_dir, "ref_tree.code")

# get the species/organism list of names used by the main.nf pipeline
species_name_file = file.path(parent_of_all_dir, "orthologs_org_ids_to_concatenate")


# each run of main.nf has one output directory. here we get that or multiple runs outputs directories.
run_dirs = Sys.glob(dir_containing_all_pipe_results)
run_dirs = run_dirs[file.info(run_dirs)$isdir]

# Create the 'figures' directory if it doesn't exist. Created where this notebook is executed.
if (!dir.exists("figures")) {
    dir.create("figures")
}

# Create the 'tables' directory if it doesn't exist. Created where this notebook is executed.
if (!dir.exists("tables")) {
    dir.create("tables")
}

<a id="chapter-4-Swapped-label-vs-true-topologies"></a>

## Chapter 4 - Swapped label vs true topologies

TODO add description 

In [41]:
# later on to this file at each family iteretion a df rows will be appended. to avoid that running this cell multiple times might append to the file over and over this line is here to write the column name as well as overwrite an existing file so it is fresh.
write.table(data.frame(method = character(), group = character(), family = character(), swap_species = character(), swap_seq = character(), rf_val = numeric()),  
            file = "tables/RF_swapped_set_all_methods_vs_reference.tsv",  
            quote = FALSE, sep = "\t", row.names = FALSE, col.names = TRUE)

for (run in run_dirs) {
    print(basename(run))

    # the mapping used later on to rename species trees according to a more generic sequence name
    ref_codefile = read.table(file = reference_tree_codefile )
    
    # get all the family subdirectories in each main.nf output run dir. 
    # This will contain the 100 normal families directories + all directories of mislabeled families.
    # There are (90) combination of 10 species and 10 sequences swaps (no swap on one sequence) per family, so 9100 dirs in each group.
    fam_dirs = grep("avg_branchlen_0.7_protein_family*", list.dirs(run, recursive = FALSE), value = TRUE)
    count_fam = -1

    # read the names of the ortholog species.
    orgs = read.table(file = species_name_file)

    # setting global df that will hold family level rf values.
    df_all_rf = data.frame(
        method       = c(),
        group        = c(),
        family       = c(),
        swap_species = c(),
        swap_seq     = c(),
        rf_val       = c()
    )

    # get all unit25 paralog trees for all combination of methods ME,ML + SM,ST
    for (m in fam_dirs) {
        
        count_fam = count_fam + 1

        # print every 10 families a check on progress to screen
        if (count_fam %% 910 == 0) {
            print(m)
        }

        # work only on swapped families directories
        if (grepl('swap_', m)) {

            # each directory has two subdirectories in which the data are stored
            tree_dir = paste0(m, "/results/rerun")

            # Read the paralog trees
            #Minimun Evolution (ME) + supermatrix method (SM)
            me_sm_path = list.files(path = tree_dir, pattern = glob2rx("*supermatrix.phylip_fastme_tree.nwk"), full.names = TRUE)
            me_sm_file = file.path(me_sm_path)
            me_sm_tree = read.tree(me_sm_file)
            me_sm_tree = unroot(me_sm_tree)

            # ME + supertree method (ST)
            me_st_path = list.files(path = tree_dir, pattern = glob2rx("avg_branchlen_*_paralog_tree.nwk"), full.names = TRUE)
            me_st_file = file.path(me_st_path)
            me_st_tree = read.tree(me_st_file)
            me_st_tree = unroot(me_st_tree)

            # Maximum Likelihood (ML) + SM
            ml_sm_path = list.files(path = tree_dir, pattern = glob2rx("RAxML_bestTree*_supermatrix_raxml.nwk"), full.names = TRUE)
            ml_sm_file = file.path(ml_sm_path)
            ml_sm_tree = read.tree(ml_sm_file)
            ml_sm_tree = unroot(ml_sm_tree)

            # ML + ST
            ml_st_path = list.files(path = tree_dir, pattern = glob2rx("RAxML_bestTree*_paralog_tree.nwk"), full.names = TRUE)
            ml_st_file = file.path(ml_st_path)
            ml_st_tree = read.tree(ml_st_file)
            ml_st_tree = unroot(ml_st_tree)

            # read the reference tree associated with this family.
            ref_tree_file = file.path(paste0(strsplit(m, '_swap_')[[1]][1], '.nwk'))
            ref_tree      = read.tree(ref_tree_file)

            # Since each gene family has a predifined gene tree (ref_tree) composed of a fixed species tree pasted as leaf of the paralog tree. The tree is designed like this by choice. Each gene families varies only in the paralog tree that it has that is unique (15 tips).
            # Here we extract the paralog tree and to do so we just need to extract the subtree with all the sequences of omne species. Since each species have the same paralog tree. So we can extract the subtree of whichever species it does not matter.
            ref_subtree   = keep.tip(ref_tree, as.character(ref_codefile$V1))
    
            # we then rename the tips of the paralog tree (ref_subtree) to a more generic keyname. Effectively going from rat1 to seq1. 
            for (j in 1:length(ref_subtree$tip.label)) { 
                ref_subtree$tip.label[j] = as.character(ref_codefile$V2[which(ref_codefile$V1 == ref_subtree[["tip.label"]][j])])
            }
    
            # Unroot the reference tree 
            ref_subtree = unroot(ref_subtree)

            # compute the RF distance matrix, all against all. 5 tree in total.
            all_trees        = c(ref_subtree,
                                 me_sm_tree,
                                 me_st_tree,
                                 ml_sm_tree,
                                 ml_st_tree
                                )
            class(all_trees) = "multiPhylo"
            all_vs_all_rf    = RF.dist(all_trees, normalize = TRUE)

            # update the family df with rf values. the first fopur columns together will act as an ID for tht tree.
            df_all_rf = data.frame(
                        method       = c(df_all_rf$method, c("SM_ME", "ST_ME", "SM_ML", "ST_ML")),
                        group        = c(df_all_rf$group, rep(basename(run), each = 4)),
                        family       = c(df_all_rf$family, rep(strsplit(basename(m), '_swap_')[[1]][1], each = 4)),
                        swap_species = c(df_all_rf$swap_species, rep(strsplit(m, "_")[[1]][length(strsplit(m, "_")[[1]])-1], each = 4)),      # getting the number of species affected by the swap
                        swap_seq     = c(df_all_rf$swap_seq, rep(strsplit(m, "_")[[1]][length(strsplit(m, "_")[[1]])], each = 4)),  # getting the number of sequences swapped per species
                        rf_val       = c(df_all_rf$rf_val,  c(all_vs_all_rf[1], all_vs_all_rf[2], all_vs_all_rf[3], all_vs_all_rf[4]))
            )

        # every time a non combination dir is encountered write to file what has been stored up untill now and reset the df
        } else {

            # appending the table with family rf values to the file. done this way so it is faster
            write.table(df_all_rf, file = "tables/RF_swapped_set_all_methods_vs_reference.tsv", quote = FALSE, sep = "\t", row.names = FALSE, col.names = FALSE, append = TRUE)

            # empty df
            df_all_rf = data.frame(
                method       = c(),
                group        = c(),
                family       = c(),
                swap_species = c(),
                swap_seq     = c(),
                rf_val       = c()
            )
        }
    }

    # appending the last family combinations of that group
    write.table(df_all_rf, file = "tables/RF_swapped_set_all_methods_vs_reference.tsv", quote = FALSE, sep = "\t", row.names = FALSE, col.names = FALSE, append = TRUE)

} 

[1] "group1_23th"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/swapped_data/group1_23th/avg_branchlen_0.7_protein_family_001.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/swapped_data/group1_23th/avg_branchlen_0.7_protein_family_011.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/swapped_data/group1_23th/avg_branchlen_0.7_protein_family_021.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/swapped_data/group1_23th/avg_branchlen_0.7_protein_family_031.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/swapped_data/group1_23th/avg_branchlen_0.7_protein_family_041.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_correc

<a id="chapter-5-RF-tables-report"></a>

## Chapter 5 - RF tables report

TODO add description 

In [43]:
# group 1, ME + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group1_ME_SM = df_new  %>%
  filter(group == "group1_23th", method == "SM_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group1_ME_SM_processed = df_group1_ME_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group1_ME_SM_processed

swap_species,2,3,4,5,6,7,8,9,10
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.347 ± 0.15,0.341 ± 0.148,0.344 ± 0.152,0.34 ± 0.151,0.343 ± 0.155,0.331 ± 0.145,0.341 ± 0.15,0.335 ± 0.15,0.338 ± 0.163
2,0.343 ± 0.147,0.346 ± 0.149,0.337 ± 0.146,0.326 ± 0.147,0.35 ± 0.151,0.337 ± 0.149,0.341 ± 0.143,0.338 ± 0.151,0.334 ± 0.15
3,0.338 ± 0.152,0.346 ± 0.151,0.339 ± 0.148,0.348 ± 0.151,0.34 ± 0.15,0.344 ± 0.157,0.341 ± 0.156,0.339 ± 0.153,0.332 ± 0.15
4,0.335 ± 0.152,0.345 ± 0.156,0.337 ± 0.149,0.334 ± 0.152,0.348 ± 0.154,0.342 ± 0.144,0.348 ± 0.167,0.327 ± 0.154,0.346 ± 0.159
5,0.341 ± 0.146,0.343 ± 0.162,0.342 ± 0.154,0.34 ± 0.159,0.338 ± 0.156,0.341 ± 0.159,0.346 ± 0.147,0.328 ± 0.143,0.334 ± 0.16
6,0.344 ± 0.161,0.337 ± 0.144,0.332 ± 0.148,0.339 ± 0.144,0.333 ± 0.152,0.345 ± 0.165,0.33 ± 0.153,0.329 ± 0.159,0.334 ± 0.16
7,0.342 ± 0.148,0.333 ± 0.16,0.327 ± 0.138,0.322 ± 0.161,0.332 ± 0.151,0.333 ± 0.153,0.335 ± 0.144,0.342 ± 0.169,0.333 ± 0.154
8,0.335 ± 0.146,0.333 ± 0.148,0.328 ± 0.147,0.347 ± 0.137,0.339 ± 0.151,0.335 ± 0.165,0.313 ± 0.162,0.338 ± 0.144,0.328 ± 0.162
9,0.341 ± 0.155,0.34 ± 0.154,0.341 ± 0.149,0.346 ± 0.146,0.345 ± 0.148,0.331 ± 0.148,0.35 ± 0.165,0.344 ± 0.158,0.332 ± 0.148


In [44]:
# group 1, ME + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + ST 
df_group1_ME_ST = df_new  %>%
  filter(group == "group1_23th", method == "ST_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group1_ME_ST_processed = df_group1_ME_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)


df_group1_ME_ST_processed

swap_species,2,3,4,5,6,7,8,9,10
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.356 ± 0.161,0.353 ± 0.163,0.349 ± 0.16,0.35 ± 0.161,0.352 ± 0.16,0.353 ± 0.159,0.348 ± 0.161,0.354 ± 0.163,0.349 ± 0.158
2,0.351 ± 0.161,0.35 ± 0.163,0.349 ± 0.162,0.348 ± 0.158,0.354 ± 0.16,0.353 ± 0.158,0.352 ± 0.161,0.352 ± 0.159,0.354 ± 0.165
3,0.352 ± 0.163,0.352 ± 0.16,0.356 ± 0.165,0.358 ± 0.159,0.356 ± 0.167,0.354 ± 0.157,0.35 ± 0.16,0.355 ± 0.161,0.348 ± 0.153
4,0.354 ± 0.162,0.353 ± 0.158,0.353 ± 0.154,0.358 ± 0.158,0.352 ± 0.167,0.353 ± 0.166,0.351 ± 0.155,0.349 ± 0.163,0.353 ± 0.161
5,0.354 ± 0.163,0.352 ± 0.158,0.356 ± 0.164,0.351 ± 0.164,0.35 ± 0.158,0.356 ± 0.168,0.353 ± 0.163,0.357 ± 0.159,0.355 ± 0.156
6,0.358 ± 0.161,0.344 ± 0.149,0.351 ± 0.157,0.355 ± 0.162,0.357 ± 0.158,0.35 ± 0.158,0.358 ± 0.159,0.348 ± 0.162,0.349 ± 0.171
7,0.357 ± 0.159,0.354 ± 0.163,0.352 ± 0.159,0.352 ± 0.16,0.353 ± 0.154,0.353 ± 0.166,0.355 ± 0.161,0.352 ± 0.162,0.358 ± 0.156
8,0.354 ± 0.163,0.356 ± 0.165,0.354 ± 0.16,0.358 ± 0.163,0.343 ± 0.157,0.355 ± 0.163,0.344 ± 0.167,0.358 ± 0.153,0.348 ± 0.15
9,0.349 ± 0.157,0.348 ± 0.16,0.357 ± 0.158,0.348 ± 0.156,0.353 ± 0.156,0.355 ± 0.161,0.356 ± 0.166,0.358 ± 0.16,0.351 ± 0.162


In [46]:
# group 1, ML + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group1_ML_SM = df_new  %>%
  filter(group == "group1_23th", method == "SM_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group1_ML_SM_processed = df_group1_ML_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group1_ML_SM_processed

swap_species,2,3,4,5,6,7,8,9,10
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.272 ± 0.155,0.271 ± 0.148,0.269 ± 0.148,0.277 ± 0.154,0.272 ± 0.149,0.277 ± 0.151,0.265 ± 0.154,0.282 ± 0.149,0.268 ± 0.15
2,0.277 ± 0.154,0.276 ± 0.155,0.27 ± 0.158,0.268 ± 0.156,0.28 ± 0.162,0.262 ± 0.144,0.275 ± 0.147,0.272 ± 0.157,0.282 ± 0.153
3,0.269 ± 0.162,0.269 ± 0.146,0.273 ± 0.156,0.284 ± 0.156,0.27 ± 0.147,0.287 ± 0.158,0.274 ± 0.149,0.269 ± 0.146,0.268 ± 0.144
4,0.273 ± 0.154,0.277 ± 0.16,0.278 ± 0.154,0.278 ± 0.146,0.283 ± 0.148,0.271 ± 0.154,0.267 ± 0.154,0.264 ± 0.156,0.272 ± 0.147
5,0.274 ± 0.15,0.272 ± 0.147,0.262 ± 0.153,0.269 ± 0.149,0.269 ± 0.145,0.277 ± 0.16,0.272 ± 0.15,0.27 ± 0.154,0.273 ± 0.153
6,0.268 ± 0.142,0.285 ± 0.154,0.273 ± 0.136,0.284 ± 0.132,0.27 ± 0.145,0.273 ± 0.155,0.27 ± 0.15,0.28 ± 0.148,0.275 ± 0.146
7,0.284 ± 0.159,0.268 ± 0.147,0.273 ± 0.152,0.258 ± 0.139,0.267 ± 0.148,0.287 ± 0.164,0.268 ± 0.143,0.285 ± 0.162,0.265 ± 0.142
8,0.265 ± 0.149,0.274 ± 0.144,0.279 ± 0.153,0.283 ± 0.148,0.277 ± 0.147,0.276 ± 0.159,0.268 ± 0.152,0.282 ± 0.153,0.266 ± 0.143
9,0.275 ± 0.156,0.276 ± 0.144,0.286 ± 0.156,0.279 ± 0.144,0.283 ± 0.149,0.274 ± 0.16,0.273 ± 0.154,0.288 ± 0.153,0.296 ± 0.158


In [45]:
# group 5, ME + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group5_ME_SM = df_new  %>%
  filter(group == "group5_11th", method == "SM_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group5_ME_SM_processed = df_group5_ME_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group5_ME_SM_processed

swap_species,2,3,4,5,6,7,8,9,10
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.143 ± 0.103,0.144 ± 0.102,0.155 ± 0.105,0.148 ± 0.1,0.15 ± 0.108,0.143 ± 0.105,0.148 ± 0.112,0.153 ± 0.106,0.141 ± 0.104
2,0.148 ± 0.104,0.147 ± 0.104,0.153 ± 0.105,0.141 ± 0.099,0.154 ± 0.099,0.154 ± 0.11,0.149 ± 0.11,0.152 ± 0.103,0.148 ± 0.105
3,0.147 ± 0.109,0.153 ± 0.106,0.16 ± 0.112,0.147 ± 0.103,0.152 ± 0.105,0.143 ± 0.1,0.142 ± 0.102,0.146 ± 0.112,0.153 ± 0.107
4,0.154 ± 0.111,0.148 ± 0.105,0.141 ± 0.1,0.138 ± 0.103,0.15 ± 0.109,0.144 ± 0.104,0.143 ± 0.109,0.15 ± 0.103,0.155 ± 0.112
5,0.146 ± 0.105,0.151 ± 0.106,0.152 ± 0.096,0.147 ± 0.103,0.146 ± 0.1,0.152 ± 0.111,0.148 ± 0.105,0.145 ± 0.101,0.156 ± 0.094
6,0.158 ± 0.113,0.136 ± 0.095,0.156 ± 0.108,0.147 ± 0.105,0.147 ± 0.105,0.147 ± 0.104,0.148 ± 0.1,0.175 ± 0.108,0.152 ± 0.108
7,0.143 ± 0.105,0.161 ± 0.107,0.151 ± 0.1,0.151 ± 0.101,0.155 ± 0.109,0.148 ± 0.106,0.168 ± 0.111,0.148 ± 0.102,0.154 ± 0.104
8,0.152 ± 0.102,0.148 ± 0.108,0.158 ± 0.104,0.15 ± 0.108,0.148 ± 0.106,0.147 ± 0.107,0.159 ± 0.121,0.15 ± 0.11,0.155 ± 0.116
9,0.152 ± 0.099,0.157 ± 0.104,0.153 ± 0.104,0.161 ± 0.11,0.154 ± 0.107,0.158 ± 0.109,0.16 ± 0.111,0.156 ± 0.103,0.156 ± 0.112


In [47]:
# group 5, ML + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group5_ML_SM = df_new  %>%
  filter(group == "group5_11th", method == "SM_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group5_ML_SM_processed = df_group5_ML_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group5_ML_SM_processed

swap_species,2,3,4,5,6,7,8,9,10
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.11 ± 0.084,0.106 ± 0.081,0.107 ± 0.083,0.106 ± 0.088,0.108 ± 0.086,0.111 ± 0.08,0.107 ± 0.082,0.103 ± 0.083,0.108 ± 0.084
2,0.116 ± 0.087,0.104 ± 0.08,0.103 ± 0.086,0.112 ± 0.084,0.104 ± 0.085,0.108 ± 0.085,0.109 ± 0.085,0.108 ± 0.082,0.111 ± 0.085
3,0.107 ± 0.083,0.113 ± 0.09,0.113 ± 0.088,0.104 ± 0.085,0.108 ± 0.086,0.111 ± 0.085,0.111 ± 0.086,0.112 ± 0.09,0.108 ± 0.092
4,0.107 ± 0.087,0.109 ± 0.095,0.11 ± 0.085,0.111 ± 0.085,0.117 ± 0.088,0.113 ± 0.093,0.119 ± 0.094,0.114 ± 0.102,0.112 ± 0.094
5,0.109 ± 0.083,0.113 ± 0.09,0.113 ± 0.092,0.114 ± 0.091,0.111 ± 0.089,0.111 ± 0.089,0.119 ± 0.093,0.118 ± 0.095,0.113 ± 0.09
6,0.111 ± 0.089,0.107 ± 0.08,0.109 ± 0.092,0.11 ± 0.085,0.118 ± 0.096,0.12 ± 0.089,0.12 ± 0.094,0.124 ± 0.096,0.113 ± 0.084
7,0.108 ± 0.091,0.114 ± 0.093,0.117 ± 0.087,0.111 ± 0.089,0.124 ± 0.09,0.108 ± 0.091,0.128 ± 0.091,0.118 ± 0.099,0.129 ± 0.099
8,0.106 ± 0.087,0.114 ± 0.096,0.116 ± 0.082,0.114 ± 0.095,0.123 ± 0.097,0.113 ± 0.095,0.121 ± 0.092,0.118 ± 0.096,0.121 ± 0.091
9,0.113 ± 0.09,0.117 ± 0.089,0.109 ± 0.089,0.12 ± 0.094,0.121 ± 0.094,0.13 ± 0.096,0.13 ± 0.098,0.128 ± 0.097,0.139 ± 0.099


In [48]:
# group 5, ML + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group5_ML_ST = df_new  %>%
  filter(group == "group5_11th", method == "ST_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group5_ML_ST_processed = df_group5_ML_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group5_ML_ST_processed

swap_species,2,3,4,5,6,7,8,9,10
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.119 ± 0.086,0.115 ± 0.082,0.113 ± 0.084,0.11 ± 0.084,0.114 ± 0.089,0.115 ± 0.086,0.111 ± 0.082,0.112 ± 0.082,0.114 ± 0.083
2,0.115 ± 0.085,0.109 ± 0.081,0.109 ± 0.08,0.113 ± 0.083,0.116 ± 0.084,0.112 ± 0.089,0.115 ± 0.083,0.116 ± 0.088,0.113 ± 0.081
3,0.117 ± 0.087,0.113 ± 0.087,0.111 ± 0.082,0.112 ± 0.085,0.117 ± 0.089,0.113 ± 0.086,0.116 ± 0.083,0.117 ± 0.086,0.114 ± 0.091
4,0.113 ± 0.085,0.114 ± 0.083,0.113 ± 0.082,0.111 ± 0.088,0.115 ± 0.081,0.121 ± 0.09,0.121 ± 0.088,0.117 ± 0.09,0.115 ± 0.086
5,0.112 ± 0.082,0.115 ± 0.084,0.113 ± 0.081,0.113 ± 0.078,0.115 ± 0.084,0.109 ± 0.082,0.113 ± 0.083,0.117 ± 0.085,0.108 ± 0.079
6,0.115 ± 0.086,0.114 ± 0.082,0.113 ± 0.088,0.112 ± 0.081,0.115 ± 0.085,0.112 ± 0.082,0.111 ± 0.087,0.113 ± 0.09,0.105 ± 0.08
7,0.111 ± 0.088,0.112 ± 0.079,0.116 ± 0.084,0.115 ± 0.082,0.111 ± 0.078,0.113 ± 0.088,0.117 ± 0.088,0.112 ± 0.087,0.111 ± 0.087
8,0.113 ± 0.087,0.115 ± 0.085,0.108 ± 0.08,0.11 ± 0.079,0.113 ± 0.082,0.109 ± 0.078,0.112 ± 0.083,0.111 ± 0.086,0.121 ± 0.089
9,0.112 ± 0.086,0.114 ± 0.086,0.116 ± 0.089,0.115 ± 0.089,0.113 ± 0.084,0.114 ± 0.084,0.116 ± 0.088,0.122 ± 0.09,0.124 ± 0.087
